Importing Library

In [1]:
import cv2
import csv
import numpy as np
from follower import *


Capturing Video and initialize tracking 

In [2]:
follower = track()
cap = cv2.VideoCapture('cars2.mp4')
input = 320

CocoDataset on Yolov3-320

In [3]:
clsname = open("coco.names").read().strip().split('\n')
print(clsname)
print(len(clsname))

req_clss_index = [2, 3, 5, 7]

detected_clsname = []

modelConfiguration = 'yolov3-320.cfg'
modelWeigheights = 'yolov3-320.weights'

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeigheights)


['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
80


In [4]:
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(clsname), 3), dtype='uint8')

Finding the points of the detected Vehicals 

In [5]:
def Point_representation(value_x, value_y, value_width, value_height):
    x1=int(value_width/2)
    y1=int(value_height/2)
    cx = value_x+x1
    cy=value_y+y1
    return cx, cy

Making list of vehicals passing through the center line 

In [6]:
temp_Vehicals_going_up = []
temp_Vehicals_going_down = []
Vehicals_going_up = [0, 0, 0, 0]
Vehicals_going_down = [0, 0, 0, 0]

Function to find and append vehical going up or down and updating the list 

In [7]:
def count_vehicle(box_id, frame_taken):

    value_x, value_y, value_width, value_height, id, index = box_id

    center = Point_representation(value_x, value_y, value_width, value_height)
    ix, iy = center

    if (iy > 240) and (iy < 225):

        if id not in temp_Vehicals_going_up:
            temp_Vehicals_going_up.append(id)

    elif iy < 210 and iy > 225:
        if id not in temp_Vehicals_going_down:
            temp_Vehicals_going_down.append(id)
            
    elif iy < 240:
        if id in temp_Vehicals_going_down:
            temp_Vehicals_going_down.remove(id)
            Vehicals_going_up[index] = Vehicals_going_up[index]+1

    elif iy > 210:
        if id in temp_Vehicals_going_up:
            temp_Vehicals_going_up.remove(id)
            Vehicals_going_down[index] = Vehicals_going_down[index] + 1

    cv2.circle(frame_taken, center, 2, (0, 0, 255), -1)  


Post processing

In [8]:
def postProcess(outputs,frame_taken):
    global detected_clsname 
    height, width = frame_taken.shape[:2]
    boxes = []
    classIds = []
    conf_scr = []
    detection = []
    for output in outputs:
        for det in output:
            scr = det[5:]
            classId = np.argmax(scr)
            conf = scr[classId]
            if classId in req_clss_index:
                if conf > 0.2:
                    # print(classId)
                    value_width,value_height = int(det[2]*width) , int(det[3]*height)
                    value_x,value_y = int((det[0]*width)-value_width/2) , int((det[1]*height)-value_height/2)
                    boxes.append([value_x,value_y,value_width,value_height])
                    classIds.append(classId)
                    conf_scr.append(float(conf))

    indices = cv2.dnn.NMSBoxes(boxes, conf_scr, 0.2, 0.2)
   
    for i in indices.flatten():
        value_x, value_y, value_width, value_height = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]

        color = [int(c) for c in colors[classIds[i]]]
        name = clsname[classIds[i]]
        detected_clsname.append(name)

        cv2.putText(frame_taken,f'{name.upper()} {int(conf_scr[i]*100)}%',
                  (value_x, value_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        cv2.rectangle(frame_taken, (value_x, value_y), (value_x + value_width, value_y + value_height), color, 1)
        detection.append([value_x, value_y, value_width, value_height, req_clss_index.index(classIds[i])])

    boxes_ids = follower.update(detection)
    for box_id in boxes_ids:
        count_vehicle(box_id, frame_taken)


Annotations on the video

In [9]:
def video_run():
    while True:
        success, frame_taken = cap.read()
        frame_taken = cv2.resize(frame_taken,(0,0),None,0.5,0.5)
        ih, iw, channels = frame_taken.shape
        blob = cv2.dnn.blobFromImage(frame_taken, 1 / 255, (input, input), [0, 0, 0], 1, crop=False)

        net.setInput(blob)
        layersNames = net.getLayerNames()
        outputNames = [(layersNames[i - 1]) for i in net.getUnconnectedOutLayers()]

        outputs = net.forward(outputNames)
    
       
        postProcess(outputs,frame_taken)


        cv2.line(frame_taken, (0, 225), (iw, 225), (255, 0, 255), 2)
        cv2.line(frame_taken, (0, 240), (iw, 240), (0, 110, 255), 1)
        cv2.line(frame_taken, (0, 210), (iw, 210), (0, 110, 255),1 )

        cv2.putText(frame_taken, "Up", (110, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        cv2.putText(frame_taken, "Down", (160, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        cv2.putText(frame_taken, "Car:        "+str(Vehicals_going_up[0])+"     "+ str(Vehicals_going_down[0]), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        cv2.putText(frame_taken, "Motorbike:  "+str(Vehicals_going_up[1])+"     "+ str(Vehicals_going_down[1]), (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        cv2.putText(frame_taken, "Bus:        "+str(Vehicals_going_up[2])+"     "+ str(Vehicals_going_down[2]), (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        cv2.putText(frame_taken, "Truck:      "+str(Vehicals_going_up[3])+"     "+ str(Vehicals_going_down[3]), (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        cv2.imshow('Output', frame_taken)

        if cv2.waitKey(1) == ord('q'):
            break

    with open("data.csv", 'w') as f1:
        csvedit = csv.writer(f1)
        csvedit.writerow(['Direction', 'car', 'motorbike', 'bus', 'truck'])
        Vehicals_going_up.insert(0, "Up")
        Vehicals_going_down.insert(0, "Down")
        csvedit.writerow(Vehicals_going_up)
        csvedit.writerow(Vehicals_going_down)
    f1.close()
    cap.release()
    cv2.destroyAllWindows()

In [10]:
video_run()